In [12]:
import pandas as pd
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()
file_path = os.getenv('file_path')

In [2]:
md_token = os.getenv('md_token')
con = duckdb.connect("md:product_sales_db", config={"motherduck_token": md_token})

In [3]:
product_sales = con.sql(
    """
    select
    *
    from product_sales.v_mart_product_sales
    """
).df()
product_sales.head()

,order_id,customer_id,product,price_each,qty_ordered,total_price,order_time_stamp,order_date_part,year,month,...,day_name_num,day_month,purchase_address,city,state_name,capital,latitude,longitude,store,payment_method
0,147268,125160,Wired Headphones,11.99,1,11.99,2019-01-01 03:07:00,2019-01-01,2019,1,...,2,1,"9 Lake St, New York City, NY 10001",New York City,New York,Albany,35.78043000000000,-78.63909900000000,Walmart,Credit
1,148041,103419,USB-C Charging Cable,11.95,1,11.95,2019-01-01 03:40:00,2019-01-01,2019,1,...,2,1,"760 Church St, San Francisco, CA 94016",San Francisco,California,Sacramento,38.57666800000000,-121.49362900000000,eBay,Credit
2,149343,99418,Apple Airpods Headphones,150.00,1,150.00,2019-01-01 04:56:00,2019-01-01,2019,1,...,2,1,"735 5th St, New York City, NY 10001",New York City,New York,Albany,35.78043000000000,-78.63909900000000,eBay,Credit
3,149964,101715,AAA Batteries (4-pack),2.99,1,2.99,2019-01-01 05:53:00,2019-01-01,2019,1,...,2,1,"75 Jackson St, Dallas, TX 75001",Dallas,Texas,Austin,30.27467000000000,-97.74034900000000,eBay,Credit
4,149350,132005,USB-C Charging Cable,11.95,2,23.90,2019-01-01 06:03:00,2019-01-01,2019,1,...,2,1,"943 2nd St, Atlanta, GA 30301",Atlanta,Georgia,Atlanta,30.43811800000000,-84.28129600000000,Target,Cash


## Cast latitude and longitude to proper data format
Current format is in string/varchar. Duckdb rounds off decimal values into 2 decimal places which affects the accuracy of the coordinates.

In [4]:
product_sales['latitude'] = product_sales['latitude'].astype(float).round(6)
product_sales['longitude'] = product_sales['longitude'].astype(float).round(6)

In [5]:
product_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185652 entries, 0 to 185651
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   order_id          185652 non-null  int32         
 1   customer_id       185652 non-null  int32         
 2   product           185652 non-null  object        
 3   price_each        185652 non-null  float64       
 4   qty_ordered       185652 non-null  int32         
 5   total_price       185652 non-null  float64       
 6   order_time_stamp  185652 non-null  datetime64[us]
 7   order_date_part   185652 non-null  datetime64[us]
 8   year              185652 non-null  int32         
 9   month             185652 non-null  int32         
 10  month_name        185652 non-null  object        
 11  day_name          185652 non-null  object        
 12  day_name_num      185652 non-null  int32         
 13  day_month         185652 non-null  int32         
 14  purc

# Save as parquet and csv

In [13]:
filename= 'product_sales_2019.parquet'
root_path = os.getenv('root_path')
path = root_path + filename
product_sales.to_parquet(path,compression='gzip')